In [1]:
# read user, item and rating from the raw files
import pandas as pd
import pickle
import numpy as np

# VideoGame_df = pd.read_csv('./ratings_Video_Games.csv', names=['user_id', 'item_id', 'rating', 'time'])
# VideoGame_df['genre'] = 'VideoGame'

Toy_df = pd.read_csv('./ratings_Toys_and_Games.csv', names=['user_id', 'item_id', 'rating', 'time'])
Toy_df['genre'] = 'Toy'

# Tool_df = pd.read_csv('./ratings_Tools_and_Home_Improvement.csv', names=['user_id', 'item_id', 'rating', 'time'])
# Tool_df['genre'] = 'Tool'

# Pet_df = pd.read_csv('./ratings_Pet_Supplies.csv', names=['user_id', 'item_id', 'rating', 'time'])
# Pet_df['genre'] = 'Pet'

# Office_df = pd.read_csv('./ratings_Office_Products.csv', names=['user_id', 'item_id', 'rating', 'time'])
# Office_df['genre'] = 'Office'

# Beauty_df = pd.read_csv('./ratings_Beauty.csv', names=['user_id', 'item_id', 'rating', 'time'])
# Beauty_df['genre'] = 'Beauty'

Grocery_df = pd.read_csv('./ratings_Grocery_and_Gourmet_Food.csv', names=['user_id', 'item_id', 'rating', 'time'])
Grocery_df['genre'] = 'Grocery'

rdf = pd.concat([Grocery_df, Toy_df], sort=False)
rdf.drop(columns=['time'], inplace=True)
rdf.reset_index(drop=True, inplace=True)

In [2]:
rdf.reset_index(drop=True, inplace=True)
item_list = rdf['item_id'].unique()
user_list = rdf['user_id'].unique()
print('item num = ' + str(len(item_list)))
print('user num = ' + str(len(user_list)))

item num = 493746
user num = 1974933


In [23]:
# iteratively remove items and users with less than 10 reviews
rdf['user_freq'] = rdf.groupby('user_id')['user_id'].transform('count')
rdf.drop(rdf.index[rdf['user_freq'] <= 7], inplace=True)
rdf.reset_index(drop=True, inplace=True)
rdf['item_freq'] = rdf.groupby('item_id')['item_id'].transform('count')
rdf.drop(rdf.index[rdf['item_freq'] <= 7], inplace=True)
rdf.reset_index(drop=True, inplace=True)
rdf['user_freq'] = rdf.groupby('user_id')['user_id'].transform('count')
rdf['user_id'].value_counts()

A1UQBFCERIP7VJ    167
A2QDOJFFLFGF18    163
AAA0TUKS5VBSA     160
ACJT8MUC0LRF0     157
A3OXHLG6DIBRW8    152
ALNFHVS3SC4FV     146
A1ITRGMT80D5TK    143
A36MP37DITBU6F    125
A1W415JP5WEAJK    122
AEL6CQNQXONBX     118
A3VI2VETB90ZG5    115
ALQ4USPEQ9L5N     111
A2V92F5R7MLCVI    108
A16Z2OECAUX8Z4    108
A3NHUQ33CFH3VM    107
AGEKVD8JPZQMT     106
A2RX62V4E2BF5Z    106
AY3D7DG5L5WCK     106
AKMEY1BSHSDG7     104
A25C2M3QF9G7OQ    104
A2P739KOM4U5JB    104
A231LBC8EGPO5L    104
A353U0L2HAMSHV    103
A1II2ZRPKZAQQD    101
A1Z54EM24Y40LL    101
ALDAF4VVLFRHP      99
A24RCBRDXRXR0Y     99
A2OCDK0BOW6UCY     99
AYB4ELCS5AM8P      98
A22CW0ZHY3NJH8     97
                 ... 
A20G8KX4NPD2SF      8
A3JJNGCMI5DJGO      8
A35V8JJBYADQB9      8
A30ARMF3JEKEJY      8
A1P5QZJB3QWL0B      8
AJS7THAS9CZ59       8
A3KJ6JAZPH382D      8
AAIU4V0FTRZ2N       8
AL0MLF4AINGY3       8
ABKNRVFFLEA3M       8
A19IDPU990VUBB      8
A1WT3WVA0IJ5OO      8
A1OQJ1C1NQOABK      8
A8QITLROF01FW       8
A17CQORLF0

In [24]:
item_list = rdf['item_id'].unique()
user_list = rdf['user_id'].unique()
print('item num = ' + str(len(item_list)))
print('user num = ' + str(len(user_list)))

item num = 2487
user num = 3845


In [25]:
# count the number for each genre and sort
rdf.reset_index(drop=True, inplace=True)
item_genre_dict = dict()
for i in range(len(rdf)):
    item = rdf.at[i, 'item_id']
    genre = rdf.at[i, 'genre']
    if item not in item_genre_dict:
        item_genre_dict[item] = set([genre])
    else:
        if genre not in item_genre_dict[item]:
            item_genre_dict[item].add(genre)

import operator
genre_count = dict()
for l in item_genre_dict:
    for g in item_genre_dict[l]:
        if not g in genre_count:
            genre_count[g] = 1
        else:
            genre_count[g] += 1

genre_count_sorted = sorted(genre_count.items(), key=operator.itemgetter(1), reverse=True)
genre_count_sorted

[('Toy', 1307), ('Grocery', 1180)]

In [26]:
key_genre = ['Grocery', 'Toy']

# get the key_genre->item_list dict
key_genre_item = dict()
for k in key_genre:
    key_genre_item[k] = list()
for item in item_genre_dict:
    for g in item_genre_dict[item]:
        if g in key_genre:
            key_genre_item[g].append(item)
            
# collect all the items with key genres
key_item = list()
for genre in key_genre_item:
    key_item = key_item + key_genre_item[genre]
key_item_set = set(key_item)

# remove the non-key genre items in rdf
item_set = set(item_list)
nonkey_item_set = item_set - key_item_set
for item in nonkey_item_set:
    rdf.drop(rdf.index[rdf['item_id'] == item], inplace=True)

In [27]:
# get the user and item str id->int id dict
i = 0
user_id_dict = dict()
for u in user_list:
    if not u in user_id_dict:
        user_id_dict[u] = i
        i += 1
j = 0
item_id_dict = dict()
for i in item_list:
    if not i in item_id_dict:
        item_id_dict[i] = j
        j += 1

In [28]:
print('sparsity: ' + str(len(rdf) * 1.0 / (len(user_list) * len(item_list))))

sparsity: 0.00885290114578


In [22]:
# get the df of train, vali, and test set
rdf.reset_index(inplace=True, drop=True)
train_df = rdf.copy()
vali_df = rdf.copy()
test_df = rdf.copy()

train_ratio = 0.6
vali_ratio = 0.2
test_ratio = 0.2
num_all = len(rdf)
vali_idx = []
test_idx = []

test_vali_idx = []
i = 0
num_user = len(user_list)
for u in user_list:
    u_idx = train_df.index[train_df['user_id'] == u]
    idx_len = len(u_idx)
    test_len = int(idx_len * (test_ratio + vali_ratio))
    if test_len == 0:
        test_len = 1
    tmp = np.random.choice(u_idx, size=test_len, replace=False)
    test_vali_idx += tmp.tolist()
    i += 1
    if i % 5000 == 0:
        print(str(i) + '/' + str(num_user))

# tmp = (np.random.choice(range(num_all), size=(test_len+vali_len), replace=False)).tolist()
test_len = int(len(test_vali_idx) * test_ratio / (test_ratio + vali_ratio))
vali_len = int(len(test_vali_idx) - test_len)
test_idx = (np.random.choice(test_vali_idx, size=test_len, replace=False)).tolist()
vali_idx = (np.random.choice(test_vali_idx, size=vali_len, replace=False)).tolist()

test_set = set(test_idx)
vali_set = set(vali_idx)
train_set = set(range(num_all)) - test_set - vali_set
train_idx = list(train_set)
train_df.drop((test_idx + vali_idx), axis=0, inplace=True)
test_df.drop((train_idx + vali_idx), axis=0, inplace=True)
vali_df.drop((train_idx + test_idx), axis=0, inplace=True)

In [23]:
rdf.drop(columns=['rating'], inplace=True)
train_df.drop(columns=['rating'], inplace=True)
test_df.drop(columns=['rating'], inplace=True)
vali_df.drop(columns=['rating'], inplace=True)

In [24]:
# get the matrix of train, vali and test set

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
vali_df.reset_index(drop=True, inplace=True)
rdf.reset_index(drop=True, inplace=True)
train = np.zeros((len(user_list), len(item_list)))
test = np.zeros((len(user_list), len(item_list)))
vali = np.zeros((len(user_list), len(item_list)))
for r in range(len(train_df)):
    train[user_id_dict[train_df.at[r, 'user_id']], item_id_dict[train_df.at[r, 'item_id']]] = 1.0
for r in range(len(test_df)):
    test[user_id_dict[test_df.at[r, 'user_id']], item_id_dict[test_df.at[r, 'item_id']]] = 1.0
for r in range(len(vali_df)):
    vali[user_id_dict[vali_df.at[r, 'user_id']], item_id_dict[vali_df.at[r, 'item_id']]] = 1.0

In [25]:
# get the user int id-> str id list, and the same for item 
item_list = item_id_dict.keys()
item_idd_list = list()
for i in range(len(item_list)):
    item_idd_list.append('')
for item in item_id_dict:
    item_idd_list[item_id_dict[item]] = item

user_list = user_id_dict.keys()
user_idd_list = list()
for i in range(len(user_list)):
    user_idd_list.append('')
for user in user_id_dict:
    user_idd_list[user_id_dict[user]] = user
    
# get the item int id->genres list
item_idd_genre_list = list()
for i in range(len(item_idd_list)):
    item_idd_genre_list.append(item_genre_dict[item_idd_list[i]])

In [26]:
train_df.drop('user_freq', axis=1, inplace=True)
train_df.drop('item_freq', axis=1, inplace=True)
vali_df.drop('user_freq', axis=1, inplace=True)
vali_df.drop('item_freq', axis=1, inplace=True)
test_df.drop('user_freq', axis=1, inplace=True)
test_df.drop('item_freq', axis=1, inplace=True)
rdf.drop('user_freq', axis=1, inplace=True)
rdf.drop('item_freq', axis=1, inplace=True)

In [27]:
# get df for rdf, train, vali, test with int id for user and item
import copy
rating_df = copy.copy(rdf)
for i in range(len(rdf)):
    rating_df.at[i, 'user_id'] = user_id_dict[rating_df.at[i, 'user_id']]
    rating_df.at[i, 'item_id'] = item_id_dict[rating_df.at[i, 'item_id']]

training_df = copy.copy(train_df)
for i in range(len(training_df)):
    training_df.at[i, 'user_id'] = user_id_dict[training_df.at[i, 'user_id']]
    training_df.at[i, 'item_id'] = item_id_dict[training_df.at[i, 'item_id']]

valiing_df = copy.copy(vali_df)
for i in range(len(valiing_df)):
    valiing_df.at[i, 'user_id'] = user_id_dict[valiing_df.at[i, 'user_id']]
    valiing_df.at[i, 'item_id'] = item_id_dict[valiing_df.at[i, 'item_id']]

testing_df = copy.copy(test_df)
for i in range(len(testing_df)):
    testing_df.at[i, 'user_id'] = user_id_dict[testing_df.at[i, 'user_id']]
    testing_df.at[i, 'item_id'] = item_id_dict[testing_df.at[i, 'item_id']]

In [59]:
# generate the rating list for each key genre, get the genre->ratings dict
rdf.reset_index(drop=True, inplace=True)
key_genre_rating = dict()
for k in key_genre:
    key_genre_rating[k] = 0.0
for r in range(len(rdf)):
    item = rdf.at[r, 'item_id']
    gl = item_genre_dict[item]
    for k in key_genre:
        if k in gl:
            key_genre_rating[k] += 1.0

# get the item int id->genres list
genre_item_vector = dict()
for k in key_genre:
    genre_item_vector[k] = np.zeros((1, len(item_list)))
for i in range(len(item_idd_genre_list)):
    genre_list = item_idd_genre_list[i]
    for g in genre_list:
        if g in key_genre:
            genre_item_vector[g][0, i] = 1.0

In [60]:
with open("item_genre_dict.pkl", "wb") as f:
    pickle.dump(item_genre_dict, f, pickle.HIGHEST_PROTOCOL)
with open("genre_item_vector.pkl", "wb") as f:
    pickle.dump(genre_item_vector, f, pickle.HIGHEST_PROTOCOL)
with open("key_genre.pkl", "wb") as f:
    pickle.dump(key_genre, f, pickle.HIGHEST_PROTOCOL)
with open("user_id_dict.pkl", "wb") as f:
    pickle.dump(user_id_dict, f, pickle.HIGHEST_PROTOCOL)
with open("item_id_dict.pkl", "wb") as f:
    pickle.dump(item_id_dict, f, pickle.HIGHEST_PROTOCOL)
with open("rdf.pkl", "wb") as f:
    pickle.dump(rdf, f, pickle.HIGHEST_PROTOCOL)
with open("rating_df.pkl", "wb") as f:
    pickle.dump(rating_df, f, pickle.HIGHEST_PROTOCOL)
with open("training_df.pkl", "wb") as f:
    pickle.dump(training_df, f, pickle.HIGHEST_PROTOCOL)
with open("valiing_df.pkl", "wb") as f:
    pickle.dump(valiing_df, f, pickle.HIGHEST_PROTOCOL)
with open("testing_df.pkl", "wb") as f:
    pickle.dump(testing_df, f, pickle.HIGHEST_PROTOCOL)
with open("item_idd_genre_list.pkl", "wb") as f:
    pickle.dump(item_idd_genre_list, f, pickle.HIGHEST_PROTOCOL)
with open("item_idd_list.pkl", "wb") as f:
    pickle.dump(item_idd_list, f, pickle.HIGHEST_PROTOCOL)
with open("user_idd_list.pkl", "wb") as f:
    pickle.dump(user_idd_list, f, pickle.HIGHEST_PROTOCOL)
with open("key_genre_rating.pkl", "wb") as f:
    pickle.dump(key_genre_rating, f, pickle.HIGHEST_PROTOCOL)
    
with open("train.mat", "wb") as f:
    np.save(f, train)
with open("test.mat", "wb") as f:
    np.save(f, test)
with open("vali.mat", "wb") as f:
    np.save(f, vali)

In [29]:
# count the number for each genre and sort
import pickle
from operator import itemgetter
item_list = rdf['item_id'].unique()
item_genre_dict = pickle.load(open('./item_genre_dict.pkl'))
key_genre = pickle.load(open('./key_genre.pkl'))

genre_count = dict()
for i in item_list:
    gl = item_genre_dict[i]
    for g in gl:
        if g in key_genre:
            if not g in genre_count:
                genre_count[g] = 1
            else:
                genre_count[g] += 1

# with open("genre_count.pkl", "wb") as f:
#     pickle.dump(genre_count, f, pickle.HIGHEST_PROTOCOL)
                
genre_count_sorted = sorted(genre_count.items(), key=itemgetter(1), reverse=True)
genre_count_sorted

[('Toy', 1307), ('Grocery', 1180)]

In [30]:
import numpy as np
import pickle
import copy as copy
train = np.load('./train.mat')
item_idd_genre_list = pickle.load(open('./item_idd_genre_list.pkl'))
item_idd_genre_list = np.array(item_idd_genre_list)


mask = 1.0 * (train > 0)
user_genre_count = list()
for u in range(train.shape[0]):
    temp_genre_count = copy.copy(genre_count)
    mask_u = mask[u, :]
    gll = item_idd_genre_list[mask_u == 1.0]
    for gl in gll:
        for g in gl:
            if g in key_genre:
                temp_genre_count[g] -= 1
    user_genre_count.append(temp_genre_count)
# with open("user_genre_count.pkl", "wb") as f:
#     pickle.dump(user_genre_count, f, pickle.HIGHEST_PROTOCOL)

In [31]:
key_genre_rating = pickle.load(open('./key_genre_rating.pkl'))
genre_avg_like = dict()
for k in key_genre:
    genre_avg_like[k] = key_genre_rating[k] * 1.0 / genre_count[k]

In [32]:
genre_avg_like_sorted = sorted(genre_avg_like.items(), key=itemgetter(1), reverse=True)
genre_avg_like_sorted

[('Grocery', 46.34661016949153), ('Toy', 23.63045141545524)]